In [ ]:
from tensorflow.keras import layers, models, Input
from keras.layers import Input, Dense, Dropout, Flatten, Lambda
from keras.models import Model
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageFont, ImageDraw

from functions import prepare_data, euclidean_distance, contrastiveLoss

data_directory = 'sketch_small'
TARGET_SIZE = (224, 224)
EPOCHS = 10
BATCH_SIZE = 32

In [ ]:
X_train, X_test, y_train, y_test = prepare_data(data_directory, TARGET_SIZE)  
X_train_left = X_train[:, 0]
X_train_right = X_train[:, 1]
X_test_left = X_test[:, 0]
X_test_right = X_test[:, 1]

In [ ]:
def initialize_base_branch(input_shape=TARGET_SIZE):
    input = Input(shape=input_shape, name="base_input")
    x = Flatten(name="flatten_input")(input)
    x = Dense(128, activation='relu', name="first_base_dense")(x)
    x = Dropout(0.3, name="first_dropout")(x)
    x = Dense(128, activation='relu', name="second_base_dense")(x)
    x = Dropout(0.3, name="second_dropout")(x)
    x = Dense(128, activation='relu', name="third_base_dense")(x)

    #Returning a Model, with input and outputs, not just a group of layers. 
    return Model(inputs=input, outputs=x)


In [ ]:
#1
def siamese_network(input_shape=TARGET_SIZE):
    input_left = Input(shape=input_shape)
    input_right = Input(shape=input_shape)
    
    base_branch = initialize_base_branch(input_shape)
    
    encoded_left = base_branch(input_left)
    encoded_right = base_branch(input_right)
    
    L1_distance = Lambda(lambda tensors: abs(tensors[0] - tensors[1]))([encoded_left, encoded_right])
    
    prediction = Dense(1, activation='sigmoid')(L1_distance)
    
    siamese_network = Model(inputs=[input_left, input_right], outputs=prediction)
    
    return siamese_network

siamese_model = siamese_network()
siamese_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

siamese_model.fit([X_train_left, X_train_right], y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=0.2)

In [ ]:
#2
def siamese_network(input_shape=TARGET_SIZE):
    #change loss to contrasive
    imageA = Input(shape=input_shape)
    imageB = Input(shape=input_shape)

    model_build = initialize_base_branch(input_shape)
    modelA = model_build(imageA)
    modelB = model_build(imageB)

    distance = Lambda(euclidean_distance, output_shape=(1,))([modelA, modelB])
    model = Model(inputs=[imageA, imageB], outputs=distance)
    
    return model

siamese_model = siamese_network()
siamese_model.compile(optimizer='adam', loss=contrastiveLoss, metrics=['accuracy'])

siamese_model.fit([X_train_left, X_train_right], y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=0.2)